Script to produce 3 cleaned SolA datas and 3 GHI datas for certain date and certain site to demonstrate each mode

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import pytz #for timezone calculation
import math
import matplotlib.dates as md
import gc
import os
from datetime import datetime
%matplotlib qt
%matplotlib inline

def input_monthly_files(file_path, data_date_idx):
    """Open time-series D-PV data and ghi data of a certain month. Only compatible for SoLA data format.

    Args:
        file_path (str): The file location of the data
        data_date_idx (str): The month of the files in format 'YYYYMM' eg '201907'

    Returns:
        data (df): the opened time-series D-PV data
        ghi (df): the opened ghi data
        
    Originally written by Baran for VVAr Curtailment Calculation.
    """
    
    data_path = file_path + r"\processed_unsw_" + data_date_idx + '_data_raw.csv'
    data = pd.read_csv(data_path, index_col = 1)

    # Convert timestamp to local Adelaide time
    data.index = pd.to_datetime(data.index) # convert index from object type to datetime
    Adelaide_local_time = pytz.timezone('Australia/Adelaide')
    data.index = data.index.tz_localize(pytz.utc).tz_convert(Adelaide_local_time) # convert utc to local adelaide time
    data.index.rename('Timestamp', inplace = True)

    # Load GHI data
    ghi_date_idx = data_date_idx[0:4] + '_' + data_date_idx[4:]
    ghi_path = file_path + r"\sl_023034_" + ghi_date_idx +'.txt'
    ghi = pd.read_csv (ghi_path) 

    ghi['timestamp'] = pd.to_datetime(pd.DataFrame ({'year' : ghi['Year Month Day Hours Minutes in YYYY'].values, 
                                                    'month' : ghi['MM'], 
                                                    'day' : ghi['DD'], 
                                                   'hour' : ghi['HH24'], 
                                                   'minute' : ghi['MI format in Local standard time']}))
    ghi.set_index('timestamp', inplace = True)
    # Deal with the space characters (ghi is in object/string form at the moment)
    ghi['Mean global irradiance (over 1 minute) in W/sq m'] = [float(ghi_t) if ghi_t.count(' ')<= 3 else np.nan for ghi_t in ghi['Mean global irradiance (over 1 minute) in W/sq m']]
    return data, ghi

file_path = r"C:\Users\samha\Documents\CANVAS\data"


In [2]:
file_path = r"C:\Users\samha\Documents\CANVAS\data"

#sample_1: tripping_non_clear_sky_sample
c_id = 1317822057
date = '2019-09-03'
month = date[:4] + date[5:7]
data_sample_1, ghi_sample_1 = input_monthly_files(file_path, month)
date_dt = dt.datetime.strptime(date, '%Y-%m-%d').date()

date_filter = data_sample_1.index.date == date_dt
site_id_filter = data_sample_1['c_id'] == c_id
data_sample_1 = data_sample_1.loc[date_filter & site_id_filter]

date_filter = ghi_sample_1.index.date == date_dt
ghi_sample_1 = ghi_sample_1.loc[date_filter]

#vvar_sample
c_id = 1018350709
date = '2019-07-20'
month = date[:4] + date[5:7]
data_sample_2, ghi_sample_2 = input_monthly_files(file_path, month)
date_dt = dt.datetime.strptime(date, '%Y-%m-%d').date()

date_filter = data_sample_2.index.date == date_dt
site_id_filter = data_sample_2['c_id'] == c_id
data_sample_2 = data_sample_2.loc[date_filter & site_id_filter]

date_filter = ghi_sample_2.index.date == date_dt
ghi_sample_2 = ghi_sample_2.loc[date_filter]

#vwatt_sample
c_id = 466930914
date = '2019-08-14'
month = date[:4] + date[5:7]
data_sample_3, ghi_sample_3 = input_monthly_files(file_path, month)
date_dt = dt.datetime.strptime(date, '%Y-%m-%d').date()

date_filter = data_sample_3.index.date == date_dt
site_id_filter = data_sample_3['c_id'] == c_id
data_sample_3 = data_sample_3.loc[date_filter & site_id_filter]

date_filter = ghi_sample_3.index.date == date_dt
ghi_sample_3 = ghi_sample_3.loc[date_filter]



data_sample_1.to_csv(file_path + '/data_sample_1.csv')
data_sample_2.to_csv(file_path + '/data_sample_2.csv')
data_sample_3.to_csv(file_path + '/data_sample_3.csv')


ghi_sample_1.to_csv(file_path + '/ghi_sample_1.csv')
ghi_sample_2.to_csv(file_path + '/ghi_sample_2.csv')


In [6]:
#vwatt_sample_json_tim, good for polyfit example
c_id = 1165442853
date = '2020-04-13'
month = date[:4] + date[5:7]
data_sample_4, ghi_sample_4 = input_monthly_files(file_path, month)
date_dt = dt.datetime.strptime(date, '%Y-%m-%d').date()

date_filter = data_sample_4.index.date == date_dt
site_id_filter = data_sample_4['c_id'] == c_id
data_sample_4 = data_sample_4.loc[date_filter & site_id_filter]

date_filter = ghi_sample_4.index.date == date_dt
ghi_sample_4 = ghi_sample_4.loc[date_filter]

data_sample_4.to_csv(file_path + '/data_sample_4.csv')
ghi_sample_4.to_csv(file_path + '/ghi_sample_4.csv')